
    pip(3) install spacy
    python -m spacy download en_core_web_sm


In [54]:
from os import listdir
import en_core_web_sm
import datetime
import re
import json

In [55]:
def get_city_names(text):
    global nlp
    
    doc = nlp(text)
    locs = []
    
    for ent in doc.ents:
        if ent.label_ == "GPE":
            locs.append(ent.text)
            
    return locs

In [56]:
def create_chunks(file_text, size):
    global max_file_length
    
    return [file_text[i:i+size] for i in range(0, len(file_text), size)]

In [57]:
def extract_details(file_text):
    re_patterns = [
        "[ ]{0,4}Title: (.+)\n\n[ ]{0,4}Author: ([^\n]+)\n",
        "  We need your donations.\n\n\n([^\n]+)\n\nby ([^\n]+)\n\n",
        "\n\nTitle: (.+)\nAuthor: (.+)\nRelease Date: ",
        "\n\n\n\n\n\n[\d]{4}\n\n()\n\nby ()\n\n\nDramatis Personae",
    ]

    title = None
    author = None

    for pattern in re_patterns:
        try:
            result = re.search(pattern, file_text, re.DOTALL)
            title = result.group(1)
            author = result.group(2)
            break
        except:
            continue
    
    return title, author

In [61]:
nlp = en_core_web_sm.load()

now = datetime.datetime.now()


books = []

max_file_length = 100000 * 3

txt_files = [f for f in listdir() if f.endswith(".txt")]

large_files = 0
total_errors = 0
encoding_errors = 0
format_errors = 0

for file_name in txt_files:
    file_name = "617.txt"
    with open(file_name, 'r') as file:
        print("***************************************")
        print(file_name)

        try:
            file_text = file.read()
            
            title, author = extract_details(file_text)
            
            if author == None or title == None:
                format_errors += 1
                total_errors += 1
                print(file_name + "********ERROR FINDING DETAILS*****************")
                continue
            
            #print("title: {}".format(title))
            #print("author: {}".format(author))
            
            parts = create_chunks(file_text, max_file_length)
            
            if(len(file_text) >= max_file_length):
                print("---------------FILE BIG---------------")
                large_files += 1
            
            city_names = []
            
            for t in parts:
                city_names.extend(get_city_names(t))
            
            city_names = list(set(city_names))
            
            book = {
                "title":title, 
                "author":author,
                "city_names":city_names,
            }
            
            books.append(book)
            
            print(json.dumps(book))
            
        except UnicodeDecodeError:
            total_errors += 1
            encoding_errors += 1
            print(file_name + "********ENCODING ERROR*****************")
    break
            

print("big_files: {}".format(large_files))
print("total_errors: {}".format(total_errors))
print("format_errors: {}".format(total_errors))
print("encoding_errors: {}".format(total_errors))
time_diff = datetime.datetime.now() - now
print("seconds: {}".format(time_diff.total_seconds()))
print("minutes: {}".format(time_diff.total_seconds()/60))
print("NO of files: {}".format(len(txt_files)))

#print(json.dumps(books))

with open("books_processed.json", 'w') as fout: 
    fout.write(json.dumps(books))

***************************************
617.txt
title: Poems
author: Alan Seeger
{"title": "Poems", "author": "Alan Seeger", "city_names": ["Mississippi", "New York", "Ariosto", "Death", "Tampico", "Merida", "Baghdad", "Darien", "Chaucer", "Flowing", "Desire", "U.S.", "Samarcand", "Faustine", "Precede", "M.S.", "Bruges", "England", "Serbia", "Craonne", "Morocco", "the United States", "London", "N.Y.", "Fairyland", "France", "Broceliande", "Algiers", "Canto", "Musing", "Oneata", "Tenochtitlan", "North", "Belize", "the Balkan States", "Venice", "Penelope", "Hibernia", "Power", "Salt Lake City", "Mexico City", "Spain", "Montparnasse", "Staten Island", "Toulouse", "the Mexico City", "Ixtaccihuatl", "Throbbing", "America", "Baudelaire", "Paris", "the Second Empire", "Elijah", "New York '", "Belgrade", "United States", "New Hampshire", "Orizaba", "Inferno", "Shut", "the United\nStates", "Vivien", "Music", "Stood", "Sardinia", "Youth", "Proserpine", "Fools", "Mexico", "Mentana", "Quando", "Bo

In [60]:
print("done")

done


In [ ]:
from os import listdir
import re
import chardet
import pandas as pd
import datetime
from city import City
import mysql.connector
from mysql.connector.cursor import MySQLCursorPrepared

In [ ]:
now = datetime.datetime.now()

In [ ]:
df = pd.read_csv("worldcitiespop.csv")
df.head(1)

In [ ]:
df.count()[0]

In [ ]:
df = df[df["AccentCity"].notna()]
df = df[df["Latitude"].notna()]
df = df[df["Longitude"].notna()]
df.count()[0]

In [ ]:
df = df.sort_values(["Population"], ascending=False)
df = df.drop_duplicates(subset=["AccentCity"], keep="first")

In [ ]:
all_cities = set(df["AccentCity"])

In [ ]:
def extract_cities(file_text):
    word_list = re.sub("[^\w]", " ",  file_text).split()
    return list(all_cities.intersection(word_list))

In [ ]:
def extract_details(file_text):
    re_patterns = [
        "[ ]{0,4}Title: (.+)\n\n[ ]{0,4}Author: ([^\n]+)\n",
        "  We need your donations.\n\n\n([^\n]+)\n\nby ([^\n]+)\n\n",
        "\n\nTitle: (.+)\nAuthor: (.+)\nRelease Date: ",
        "\n\n\n\n\n\n[\d]{4}\n\n()\n\nby ()\n\n\nDramatis Personae",
    ]

    title = None
    author = None

    for pattern in re_patterns:
        try:
            result = re.search(pattern, file_text, re.DOTALL)
            title = result.group(1)
            author = result.group(2)
            break
        except:
            continue
    
    return title, author

In [ ]:
txt_files = [f for f in listdir() if f.endswith(".txt")]
#txt_files = [txt_files[0]]
success = 0
fail = 0

encoding_failed = []
regex_failed = []

books = []

for file_name in txt_files:
    
    with open(file_name, 'r') as file:
        
        try:
            file_text = file.read()
            
            title, author = extract_details(file_text)
            
            if author == None or title == None:
                print(file_name + "******REGEX*************************************************")
                fail += 1
                regex_failed.append(file_name)
                continue
            
            print(file_name)
            print("TITLE: " + title)
            print("AUTHOR: " + author + "\n")
            
            cities = extract_cities(file_text)
            
            
            success += 1
                
        except:
            print(file_name + "********ENCODING***********************************************")
            fail += 1
            encoding_failed.append(file_name)

print("Succes: " + str(success))
print("Fail: " + str(fail))
print("regex_errors: " + str(len(regex_failed)))
print("encoding_error: " + str(len(encoding_failed)))
print("Total: " + str(len(txt_files)))

In [ ]:
regex_failed[:10]

In [ ]:
encoding_failed[:10]

In [ ]:
time_diff = datetime.datetime.now() - now
time_diff.total_seconds() / 60

In [ ]:
c = City(1, "Berlin", 2, 2)

print(c.stringify())

In [ ]:
con = mysql.connector.connect(
    host="localhost",
    user="dbexamuser",
    database="examdb",
    passwd="dbExam.group7",
    use_pure=True,
)
con.autocommit = False
mycursor = con.cursor(cursor_class=MySQLCursorPrepared)
print(con) 

In [ ]:
def insert_cities():
    now = datetime.datetime.now()
    large_sql = ""
    counter = 1
    interval_counter = 0
    interval = 1000
    
    for i, row in df.iterrows():
        city_name = row["AccentCity"]
        city_long = row["Longitude"]
        city_lat = row["Latitude"]
        #print(city_name)
        #print(city_long)
        #print(city_lat)
        sql = """INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));"""
        #sql = "INSERT INTO authors (name) VALUES ('bob');"
        #large_sql += sql
        
        insert_values = (city_name, city_lat, city_long)
        
        #print(sql % insert_values)
        #print("city name: ")
        #print(type(city_name))
        #print(counter)
        
        mycursor.execute(sql, insert_values)
        #print(mycursor.statement)
        
        if (counter >= interval_counter + interval):
            print(mycursor.statement)
            con.commit()
            interval_counter += interval
            print(counter)
            
        counter += 1
        
    #print(large_sql)
    #mycursor.execute(large_sql, multi = True)
    con.commit()
    
    time_diff = datetime.datetime.now() - now
    print(time_diff.total_seconds())

insert_cities()

In [ ]:
print("done")

In [ ]:
con.close()
mycursor.close()

In [ ]:
#df[df["Latitude"] == 42.5833333]